In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import gc # garbage collection module to release memory usage in time
import time
import warnings

warnings.filterwarnings('ignore')

%matplotlib inline

In [2]:
os.chdir(r'C:\Users\frank\OneDrive - Escuela Superior de Economia y Negocios\2. SS 2024\ML Seminar\data\Rawdata')

# Datasets
## Firm characteristics

In [4]:
# load firm characteristics data
data_df = pd.read_csv('data_ch_sample.csv')
#data_df = pd.read_csv('data_ch_sample_94p.csv')
#data_df = pd.read_csv('data_ch_sample_32.csv')
#data_df = pd.read_csv('data_ch_sample_5716.csv')

In [5]:
data_ch = data_df.copy()

data_ch['DATE'] = pd.to_datetime(data_ch['DATE'],format='%Y%m%d')+pd.offsets.MonthEnd(0)
characteristics = list(set(data_ch.columns).difference({'permno','DATE','SHROUT','mve0','sic2','RET','prc'}))

data_ch.head()

,permno,DATE,mvel1,RET,prc,SHROUT,beta,betasq,chmom,dolvol,...,sp,nincr,baspread,ill,maxret,retvol,std_turn,zerotrade,sic2,bm
0,10001,2001-01-31,24355.500,0.012821,9.8750,2498,0.037079,0.001375,0.281788,8.395576,...,3.646263,2.0,0.020711,1.098587e-06,0.027778,0.017710,0.426715,4.200000e+00,49.0,0.868139
1,10002,2001-01-31,78332.625,0.088435,10.0000,8526,0.206346,0.042579,0.050021,8.067022,...,0.428502,5.0,0.033991,6.509871e-06,0.134328,0.054790,0.759666,4.200000e+00,60.0,0.680296
2,10012,2001-01-31,39836.000,0.500000,3.0000,20897,2.470629,6.104008,-1.170178,11.360419,...,0.172669,7.0,0.138777,9.482216e-08,0.129412,0.075671,7.007556,8.756593e-09,36.0,0.061049
3,10016,2001-01-31,379569.500,0.030726,23.0625,16964,0.449866,0.202379,0.391222,12.024414,...,0.602373,6.0,0.054578,5.643552e-08,0.070769,0.040708,8.102766,1.833562e-08,38.0,0.287808
4,10019,2001-01-31,28945.000,0.071429,3.7500,8270,2.249729,5.061279,0.203106,9.294773,...,2.811052,0.0,0.131620,3.206363e-07,0.435897,0.120324,16.163956,7.497863e-09,38.0,0.552262


In [6]:
data_ch = data_ch.drop(columns=['prc','SHROUT','mve0','sic2'])

In [7]:
%%time
# fill na with cross-sectional median
for ch in characteristics:
     data_ch[ch] = data_ch.groupby('DATE')[ch].transform(lambda x: x.fillna(x.median()))

CPU times: total: 7.2 s
Wall time: 8.77 s


In [8]:
for ch in characteristics:
     data_ch[ch] = data_ch[ch].fillna(0)
    
data_ch.columns[data_ch.isnull().sum()!=0]

Index([], dtype='object')

## Macroeconomic variables

In [9]:
stdt, nddt = 20010101, 20191231
data_ma = pd.read_csv('PredictorData2023.csv')
data_ma = data_ma[(data_ma['yyyymm']>=stdt//100)&(data_ma['yyyymm']<=nddt//100)].reset_index(drop=True)

In [10]:
# construct predictor
ma_predictors = ['dp_sp','ep_sp','bm_sp','ntis','tbl','tms','dfy','svar']
data_ma['Index'] = data_ma['Index'].str.replace(',','').astype('float64')
data_ma['dp_sp'] = data_ma['D12']/data_ma['Index']
data_ma['ep_sp'] = data_ma['E12']/data_ma['Index']
data_ma.rename({'b/m':'bm_sp'},axis=1,inplace=True)
data_ma['tms'] = data_ma['lty']-data_ma['tbl']
data_ma['dfy'] = data_ma['BAA']-data_ma['AAA']
data_ma = data_ma[['yyyymm']+ma_predictors]
data_ma['yyyymm'] = pd.to_datetime(data_ma['yyyymm'],format='%Y%m')+pd.offsets.MonthEnd(0)
data_ma.head()

,yyyymm,dp_sp,ep_sp,bm_sp,ntis,tbl,tms,dfy,svar
0,2001-01-31,0.011839,0.035490,0.150450,-0.003193,0.0515,0.0047,0.0078,0.004941
1,2001-02-28,0.012962,0.037873,0.156070,-0.006856,0.0488,0.0061,0.0077,0.002528
2,2001-03-31,0.013766,0.039161,0.133114,-0.005213,0.0442,0.0117,0.0086,0.007140
3,2001-04-30,0.012707,0.034060,0.122497,-0.002543,0.0387,0.0206,0.0087,0.007426
4,2001-05-31,0.012567,0.031592,0.120510,-0.000248,0.0362,0.0232,0.0078,0.002536


## Merge dataset

In [11]:
merged_data = pd.merge(data_ch, data_ma, left_on='DATE', right_on='yyyymm', how='left')

In [12]:
for feature in characteristics:
    for macro_var in ma_predictors:
        interaction_term_name = f'{feature}_x_{macro_var}'
        merged_data[interaction_term_name] = merged_data[feature] * merged_data[macro_var]

In [13]:
final_columns = ['DATE', 'permno', 'RET'] + characteristics + [f'{ch}_x_{mv}' for ch in characteristics for mv in ma_predictors]
final_data = merged_data[final_columns]

In [14]:
missing_values = final_data.isnull().sum()
missing_columns = missing_values[missing_values > 0]

if missing_columns.empty:
    print("No missing values in the dataset.")
else:
    print("Columns with missing values:")
    print(missing_columns)

No missing values in the dataset.


In [15]:
final_data['mve'] = final_data['mvel1']

In [16]:
cols_to_convert = final_data.columns.difference(['DATE', 'RET'])

final_data[cols_to_convert] = final_data[cols_to_convert].astype('float32')

In [17]:
from sklearn.preprocessing import MinMaxScaler

# Initialize the MinMaxScaler with feature range between -1 and 1
scaler = MinMaxScaler(feature_range=(-1, 1))

# List of columns to exclude from scaling
exclude_columns = ['DATE', 'permno', 'RET','mve']

# Select the columns that need to be scaled
columns_to_scale = final_data.columns.difference(exclude_columns)

# Apply the scaler to the selected columns
final_data[columns_to_scale] = scaler.fit_transform(final_data[columns_to_scale])

# Display the scaled data
final_data.head()

,DATE,permno,RET,beta,turn,std_turn,mom6m,ill,agr,chmom,...,mom1m_x_svar,mom36m_x_dp_sp,mom36m_x_ep_sp,mom36m_x_bm_sp,mom36m_x_ntis,mom36m_x_tbl,mom36m_x_tms,mom36m_x_dfy,mom36m_x_svar,mve
0,2001-01-31,10001.0,0.012821,-0.334353,-0.996730,-0.997841,-0.624620,-0.997955,0.738642,-0.016437,...,0.302977,-0.765574,-0.838730,-0.835249,0.235811,-0.868736,-0.826438,-0.692673,-0.623613,24355.500
1,2001-01-31,10002.0,0.088435,-0.277169,-0.998115,-0.996156,-0.740385,-0.987881,0.757405,-0.053515,...,0.307640,-0.787495,-0.859870,-0.851483,0.240010,-0.907100,-0.831367,-0.714360,-0.633696,78332.625
2,2001-01-31,10012.0,0.500000,0.487780,-0.956415,-0.964540,-0.945455,-0.999823,0.677703,-0.248726,...,0.250735,-0.709595,-0.784748,-0.793793,0.225088,-0.770768,-0.813849,-0.637291,-0.597866,39836.000
3,2001-01-31,10016.0,0.030726,-0.194900,-0.986370,-0.958998,-0.641981,-0.999895,0.770680,0.001071,...,0.319705,-0.733920,-0.808205,-0.811807,0.229748,-0.813338,-0.819319,-0.661356,-0.609054,379569.500
4,2001-01-31,10019.0,0.071429,0.413153,-0.974922,-0.918206,-0.769568,-0.999403,0.850155,-0.029024,...,0.278801,-0.798230,-0.870222,-0.859433,0.242066,-0.925888,-0.833781,-0.724981,-0.638634,28945.000


In [18]:
gc.collect()

0

In [19]:
final_data_top = final_data.sort_values('mve',ascending=False).groupby('DATE').head(1000).reset_index(drop=True)
#final_data_bot = final_data.sort_values('mve',ascending=False).groupby('DATE').tail(1000).reset_index(drop=True)

In [20]:
del([ch,characteristics,columns_to_scale,data_ch,data_ma,exclude_columns,feature,final_columns,interaction_term_name,ma_predictors,macro_var,merged_data,missing_columns,missing_values,nddt,scaler,stdt])

In [21]:
del(cols_to_convert)

In [22]:
gc.collect()

0

# Sample splitting

In [23]:
del(final_data)
data_top = final_data_top.copy()
del(final_data_top)
del(data_df)

In [24]:
gc.collect()

31

In [25]:
data_top = data_top.sort_values(by=['DATE', 'permno']).reset_index(drop=True)

Define the features

In [26]:
features = list(set(data_top.columns).difference({'permno','DATE','mve','RET'}))

Now, let's split the datasets into train, valid and test

In [27]:
# Define time frontiers for validation and test sets
stdt_vld = np.datetime64('2009-01-31')
stdt_tst = np.datetime64('2013-01-31')

# Training set: data before the validation start date
train_data = data_top[data_top['DATE'] < stdt_vld]

# Validation set: data between validation start date and test start date
valid_data = data_top[(data_top['DATE'] >= stdt_vld) & (data_top['DATE'] < stdt_tst)]

# Test set: data after the test start date
test_data = data_top[data_top['DATE'] >= stdt_tst]

In [28]:
gc.collect()

31

# Model implementation
## Performance metrics

### Define the $R_{oos}^{2}$

In [29]:
from sklearn.metrics import mean_squared_error

# Scoring Function
# out-of-sample R squared
def R_oos(actual, predicted):
    actual, predicted = np.array(actual), np.array(predicted).flatten()
    #predicted = np.where(predicted<0,0,predicted)
    return 1 - (np.dot((actual-predicted),(actual-predicted)))/(np.dot(actual,actual))

# Evaluation Output
def evaluate(actual, predicted):
    print('*'*15+'Out-of-Sample Metrics'+'*'*15)
    print(f'The out-of-sample R2 is {R_oos(actual,predicted)*100:.2f}%')
    print(f'The out-of-sample MSE is {mean_squared_error(actual,predicted):.3f}')

## OLS
### All features
Using the $l_2$ as loss function

In [30]:
%%time
time.sleep(10)
from sklearn.linear_model import LinearRegression

OLS = LinearRegression().fit(train_data[features],train_data['RET'])
evaluate(test_data['RET'], OLS.predict(test_data[features]))

***************Out-of-Sample Metrics***************
The out-of-sample R2 is -68.87%
The out-of-sample MSE is 0.011
CPU times: total: 16.4 s
Wall time: 14.8 s


Using the Huber loss function

In [31]:
%%time
time.sleep(10)
from sklearn.linear_model import HuberRegressor

epsilon = np.max(((train_data['RET']-OLS.predict(train_data[features])).quantile(.999),1))
OLS_H = HuberRegressor(epsilon=epsilon).fit(train_data[features],train_data['RET'])
evaluate(test_data['RET'], OLS_H.predict(test_data[features]))

***************Out-of-Sample Metrics***************
The out-of-sample R2 is -0.38%
The out-of-sample MSE is 0.006
CPU times: total: 1min 57s
Wall time: 1min 2s


In [32]:
gc.collect()

31

### OLS-3

In [33]:
features_3 = ['mvel1','bm','mom1m','mom6m','mom12m','mom36m']

Using $l_2$ as loss function

In [34]:
%%time
from sklearn.linear_model import LinearRegression

OLS_3 = LinearRegression().fit(train_data[features_3],train_data['RET'])
evaluate(test_data['RET'], OLS_3.predict(test_data[features_3]))

***************Out-of-Sample Metrics***************
The out-of-sample R2 is -0.59%
The out-of-sample MSE is 0.006
CPU times: total: 15.6 ms
Wall time: 47.7 ms


Using Huber loss function

In [35]:
%%time
from sklearn.linear_model import HuberRegressor

epsilon = np.max(((train_data['RET']-OLS_3.predict(train_data[features_3])).quantile(.999),1))
OLS_H_3 = HuberRegressor(epsilon=epsilon).fit(train_data[features_3],train_data['RET'])
evaluate(test_data['RET'], OLS_H_3.predict(test_data[features_3]))

***************Out-of-Sample Metrics***************
The out-of-sample R2 is 0.82%
The out-of-sample MSE is 0.006
CPU times: total: 2.62 s
Wall time: 1.22 s


In [36]:
gc.collect()

31

# PLS

In [37]:
from sklearn.cross_decomposition import PLSRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import ParameterGrid

# Define the PLS pipeline creation function
def create_pls_pipeline(n_components):
    return Pipeline([
        ('pls', PLSRegression(n_components=n_components))  # PLS for regression
    ])

# Define the parameter grid for PLS components
params = {'pls__n_components': [1,2,3,4,5]}

# Validation function for PLS
def val_fun_pls(params, X_trn, y_trn, X_vld, y_vld):
    best_ros = -float('inf')
    best_mod = None
    best_param = None
    lst_params = list(ParameterGrid(params))
    
    for param in lst_params:
        mod = create_pls_pipeline(n_components=param['pls__n_components'])
        mod.fit(X_trn, y_trn)
        y_pred = mod.predict(X_vld)
        ros = R_oos(y_vld, y_pred)
        
        if ros > best_ros:
            best_ros = ros
            best_mod = mod
            best_param = param

    print(f'Best parameters: {best_param}')
    print(f'Best R_oos: {best_ros*100:.2f}%')
    return best_mod

# Perform hyperparameter tuning for PLS
PLS_best_model = val_fun_pls(params=params, X_trn=train_data[features], y_trn=train_data['RET'], X_vld=valid_data[features], y_vld=valid_data['RET'])

# Predict using the optimal PLS model on the test set
pls_pred_os = PLS_best_model.predict(test_data[features])

# Evaluate the PLS model's performance
evaluate(test_data['RET'], pls_pred_os)

Best parameters: {'pls__n_components': 5}
Best R_oos: 4.62%
***************Out-of-Sample Metrics***************
The out-of-sample R2 is 4.79%
The out-of-sample MSE is 0.006


# PCR

In [40]:
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import mean_squared_error

# Define the PCR pipeline creation function
def create_pcr_pipeline(n_components):
    return Pipeline([
        ('scaler', StandardScaler()),  # Standardize features
        ('pca', PCA(n_components=n_components)),  # PCA for dimensionality reduction
        ('regression', LinearRegression())  # Linear regression on principal components
    ])

# Define the parameter grid for PCA components
params = {'pca__n_components': [36,37,38,39,40]}

# Validation function for PCR
def val_fun_pcr(params, X_trn, y_trn, X_vld, y_vld):
    best_ros = -float('inf')
    best_mod = None
    best_param = None
    lst_params = list(ParameterGrid(params))
    
    for param in lst_params:
        mod = create_pcr_pipeline(n_components=param['pca__n_components'])
        mod.fit(X_trn, y_trn)
        y_pred = mod.predict(X_vld)
        ros = R_oos(y_vld, y_pred)
        
        if ros > best_ros:
            best_ros = ros
            best_mod = mod
            best_param = param

    print(f'Best parameters: {best_param}')
    print(f'Best R_oos: {best_ros*100:.2f}%')
    return best_mod

# Perform hyperparameter tuning for PCR
PCR = val_fun_pcr(params=params, X_trn=train_data[features], y_trn=train_data['RET'], X_vld=valid_data[features], y_vld=valid_data['RET'])

# Predict using the optimal PCR model on the test set
pcr_pred_os = PCR.predict(test_data[features])

# Evaluate the PCR model's performance
def evaluate(actual, predicted):
    print('*'*15 + 'Out-of-Sample Metrics' + '*'*15)
    print(f'The out-of-sample R2 is {R_oos(actual, predicted)*100:.2f}%')
    print(f'The out-of-sample MSE is {mean_squared_error(actual, predicted):.3f}')

evaluate(test_data['RET'], pcr_pred_os)

Best parameters: {'pca__n_components': 38}
Best R_oos: 5.00%
***************Out-of-Sample Metrics***************
The out-of-sample R2 is 4.78%
The out-of-sample MSE is 0.006


# Neural Networks
First, import required libraries

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error

## NN1

In [ ]:
def create_nn1_model(input_dim, hidden_units):
    model = Sequential()
    # Input layer with hidden units
    model.add(Dense(hidden_units, input_dim=input_dim, activation='relu'))
    # Output layer
    model.add(Dense(1))
    return model

Compilation and training of the model

In [ ]:
def train_nn1_model(X_trn, y_trn, X_vld, y_vld, hidden_units, batch_size=32, epochs=100):
    model = create_nn1_model(X_trn.shape[1], hidden_units)
    
    # Compile the model with Adam optimizer
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
    
    # Early stopping to prevent overfitting
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    
    # Train the model
    history = model.fit(X_trn, y_trn, 
                        validation_data=(X_vld, y_vld),
                        epochs=epochs,
                        batch_size=batch_size,
                        callbacks=[early_stopping],
                        verbose=1)
    
    return model, history

Evaluation of model

In [ ]:
def evaluate_nn1_model(model, X_tst, y_tst):
    y_pred = model.predict(X_tst).flatten()
    print('*'*15 + 'Out-of-Sample Metrics' + '*'*15)
    print(f'The out-of-sample R2 is {R_oos(y_tst, y_pred)*100:.2f}%')
    print(f'The out-of-sample MSE is {mean_squared_error(y_tst, y_pred):.3f}')

Implementation on datasets

In [ ]:
# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Define architecture parameters
hidden_units = 32  # Number of neurons in the hidden layer

# Train the model
nn1_model, history = train_nn1_model(X_trn, y_trn, X_vld, y_vld, hidden_units)

# Evaluate the model
evaluate_nn1_model(nn1_model, X_tst, y_tst)

In [ ]:
gc.collect()

## NN2

In [ ]:
def create_nn2_model(input_dim, hidden_units1, hidden_units2):
    model = Sequential()
    # First hidden layer
    model.add(Dense(hidden_units1, input_dim=input_dim, activation='relu'))
    model.add(BatchNormalization())
    # Second hidden layer
    model.add(Dense(hidden_units2, activation='relu'))
    model.add(BatchNormalization())
    # Output layer
    model.add(Dense(1))
    return model

Compile and Train the Model

In [ ]:
def train_nn2_model(X_trn, y_trn, X_vld, y_vld, hidden_units1, hidden_units2, batch_size=32, epochs=100):
    model = create_nn2_model(X_trn.shape[1], hidden_units1, hidden_units2)
    
    # Compile the model with Adam optimizer
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
    
    # Early stopping to prevent overfitting
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    
    # Train the model
    history = model.fit(X_trn, y_trn, 
                        validation_data=(X_vld, y_vld),
                        epochs=epochs,
                        batch_size=batch_size,
                        callbacks=[early_stopping],
                        verbose=1)
    
    return model, history

Evaluate the Model

In [ ]:
def evaluate_nn2_model(model, X_tst, y_tst):
    y_pred = model.predict(X_tst).flatten()
    print('*'*15 + 'Out-of-Sample Metrics' + '*'*15)
    print(f'The out-of-sample R2 is {R_oos(y_tst, y_pred)*100:.2f}%')
    print(f'The out-of-sample MSE is {mean_squared_error(y_tst, y_pred):.3f}')

Implementation on datasets

In [ ]:
# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Define architecture parameters
hidden_units1 = 32  # Number of neurons in the first hidden layer
hidden_units2 = 16  # Number of neurons in the second hidden layer

# Train the model
nn2_model, history = train_nn2_model(X_trn, y_trn, X_vld, y_vld, hidden_units1, hidden_units2)

# Evaluate the model
evaluate_nn2_model(nn2_model, X_tst, y_tst)

## NN3

In [ ]:
def create_nn3_model(input_dim, hidden_units1, hidden_units2, hidden_units3):
    model = Sequential()
    # First hidden layer
    model.add(Dense(hidden_units1, input_dim=input_dim, activation='relu'))
    model.add(BatchNormalization())
    # Second hidden layer
    model.add(Dense(hidden_units2, activation='relu'))
    model.add(BatchNormalization())
    # Third hidden layer
    model.add(Dense(hidden_units3, activation='relu'))
    model.add(BatchNormalization())
    # Output layer
    model.add(Dense(1))
    return model

In [ ]:
def train_nn3_model(X_trn, y_trn, X_vld, y_vld, hidden_units1, hidden_units2, hidden_units3, batch_size=32, epochs=100):
    model = create_nn3_model(X_trn.shape[1], hidden_units1, hidden_units2, hidden_units3)
    
    # Compile the model with Adam optimizer
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
    
    # Early stopping to prevent overfitting
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    
    # Train the model
    history = model.fit(X_trn, y_trn, 
                        validation_data=(X_vld, y_vld),
                        epochs=epochs,
                        batch_size=batch_size,
                        callbacks=[early_stopping],
                        verbose=1)
    
    return model, history

In [ ]:
def evaluate_nn3_model(model, X_tst, y_tst):
    y_pred = model.predict(X_tst).flatten()
    print('*'*15 + 'Out-of-Sample Metrics' + '*'*15)
    print(f'The out-of-sample R2 is {R_oos(y_tst, y_pred)*100:.2f}%')
    print(f'The out-of-sample MSE is {mean_squared_error(y_tst, y_pred):.3f}')

In [ ]:
# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Define architecture parameters
hidden_units1 = 32  # Number of neurons in the first hidden layer
hidden_units2 = 16  # Number of neurons in the second hidden layer
hidden_units3 = 8   # Number of neurons in the third hidden layer

# Train the model
nn3_model, history = train_nn3_model(X_trn, y_trn, X_vld, y_vld, hidden_units1, hidden_units2, hidden_units3)

# Evaluate the model
evaluate_nn3_model(nn3_model, X_tst, y_tst)

# NN + Drop

In [44]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

## NN1

In [45]:
def create_nn1_model(input_dim, hidden_units):
    model = Sequential()
    model.add(Dense(hidden_units, input_dim=input_dim, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))  # Dropout for regularization
    model.add(Dense(1))  # Output layer
    return model

def train_nn1_model(X_trn, y_trn, X_vld, y_vld, hidden_units=32, batch_size=32, epochs=100):
    model = create_nn1_model(X_trn.shape[1], hidden_units)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    history = model.fit(X_trn, y_trn, 
                        validation_data=(X_vld, y_vld),
                        epochs=epochs,
                        batch_size=batch_size,
                        callbacks=[early_stopping],
                        verbose=1)
    return model, history

In [46]:
# Train the NN1 model
hidden_units = 32  # Number of neurons in the hidden layer
NN1_model, NN1_history = train_nn1_model(train_data[features], train_data['RET'], valid_data[features], valid_data['RET'], hidden_units=hidden_units)

# Make predictions and evaluate
NN1_predictions = NN1_model.predict(test_data[features])
evaluate(test_data['RET'], NN1_predictions)

Epoch 1/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 17s 4ms/step - loss: 0.2620 - val_loss: 0.0110
Epoch 2/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 13s 4ms/step - loss: 0.0112 - val_loss: 0.0122
Epoch 3/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 13s 4ms/step - loss: 0.0110 - val_loss: 0.0104
Epoch 4/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 13s 4ms/step - loss: 0.0108 - val_loss: 0.0113
Epoch 5/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 13s 4ms/step - loss: 0.0104 - val_loss: 0.0102
Epoch 6/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step - loss: 0.0104 - val_loss: 0.0116
Epoch 7/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step - loss: 0.0104 - val_loss: 0.0119
Epoch 8/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step - loss: 0.0105 - val_loss: 0.0119
Epoch 9/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step - loss: 0.0103 - val_loss: 0.0156
Epoch 10/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step - loss: 0.0104 - val_loss: 0.0123
Epoch 11/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step - loss: 0.0105 - val_loss: 0.0172
Epoch 12

## NN2

In [47]:
def create_nn2_model(input_dim, hidden_units1, hidden_units2):
    model = Sequential()
    model.add(Dense(hidden_units1, input_dim=input_dim, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))  # Dropout for regularization
    
    model.add(Dense(hidden_units2, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))  # Dropout for regularization
    
    model.add(Dense(1))  # Output layer
    return model

def train_nn2_model(X_trn, y_trn, X_vld, y_vld, hidden_units1=32, hidden_units2=16, batch_size=32, epochs=100):
    model = create_nn2_model(X_trn.shape[1], hidden_units1, hidden_units2)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    history = model.fit(X_trn, y_trn, 
                        validation_data=(X_vld, y_vld),
                        epochs=epochs,
                        batch_size=batch_size,
                        callbacks=[early_stopping],
                        verbose=1)
    return model, history

In [48]:
# Train the NN2 model
hidden_units1 = 32  # Number of neurons in the first hidden layer
hidden_units2 = 16  # Number of neurons in the second hidden layer
NN2_model, NN2_history = train_nn2_model(train_data[features], train_data['RET'], valid_data[features], valid_data['RET'], hidden_units1=hidden_units1, hidden_units2=hidden_units2)

# Make predictions and evaluate
NN2_predictions = NN2_model.predict(test_data[features])
evaluate(test_data['RET'], NN2_predictions)

Epoch 1/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 19s 5ms/step - loss: 0.6400 - val_loss: 0.0106
Epoch 2/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 15s 5ms/step - loss: 0.0116 - val_loss: 0.0113
Epoch 3/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 16s 5ms/step - loss: 0.0112 - val_loss: 0.0108
Epoch 4/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 18s 5ms/step - loss: 0.0116 - val_loss: 0.0123
Epoch 5/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 14s 5ms/step - loss: 0.0109 - val_loss: 0.0101
Epoch 6/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 14s 5ms/step - loss: 0.0110 - val_loss: 0.0102
Epoch 7/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 16s 5ms/step - loss: 0.0107 - val_loss: 0.0106
Epoch 8/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 15s 5ms/step - loss: 0.0108 - val_loss: 0.0106
Epoch 9/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 14s 5ms/step - loss: 0.0107 - val_loss: 0.0108
Epoch 10/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 15s 5ms/step - loss: 0.0103 - val_loss: 0.0115
Epoch 11/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 14s 5ms/step - loss: 0.0103 - val_loss: 0.0122
Epoch 12

## NN3

In [49]:
def create_nn3_model(input_dim, hidden_units1, hidden_units2, hidden_units3):
    model = Sequential()
    model.add(Dense(hidden_units1, input_dim=input_dim, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))  # Dropout for regularization
    
    model.add(Dense(hidden_units2, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))  # Dropout for regularization
    
    model.add(Dense(hidden_units3, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))  # Dropout for regularization
    
    model.add(Dense(1))  # Output layer
    return model

def train_nn3_model(X_trn, y_trn, X_vld, y_vld, hidden_units1=32, hidden_units2=16, hidden_units3=8, batch_size=32, epochs=100):
    model = create_nn3_model(X_trn.shape[1], hidden_units1, hidden_units2, hidden_units3)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    history = model.fit(X_trn, y_trn, 
                        validation_data=(X_vld, y_vld),
                        epochs=epochs,
                        batch_size=batch_size,
                        callbacks=[early_stopping],
                        verbose=1)
    return model, history

In [50]:
# Train the NN3 model
hidden_units1 = 32  # Number of neurons in the first hidden layer
hidden_units2 = 16  # Number of neurons in the second hidden layer
hidden_units3 = 8   # Number of neurons in the third hidden layer
NN3_model, NN3_history = train_nn3_model(train_data[features], train_data['RET'], valid_data[features], valid_data['RET'], hidden_units1=hidden_units1, hidden_units2=hidden_units2, hidden_units3=hidden_units3)

# Make predictions and evaluate
NN3_predictions = NN3_model.predict(test_data[features])
evaluate(test_data['RET'], NN3_predictions)

Epoch 1/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 23s 6ms/step - loss: 0.8721 - val_loss: 0.0105
Epoch 2/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 17s 6ms/step - loss: 0.0115 - val_loss: 0.0103
Epoch 3/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 17s 5ms/step - loss: 0.0115 - val_loss: 0.0103
Epoch 4/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 16s 5ms/step - loss: 0.0115 - val_loss: 0.0105
Epoch 5/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 16s 5ms/step - loss: 0.0112 - val_loss: 0.0103
Epoch 6/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 16s 5ms/step - loss: 0.0109 - val_loss: 0.0107
Epoch 7/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 16s 5ms/step - loss: 0.0109 - val_loss: 0.0136
Epoch 8/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 17s 6ms/step - loss: 0.0108 - val_loss: 0.0102
Epoch 9/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 16s 5ms/step - loss: 0.0107 - val_loss: 0.0108
Epoch 10/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 17s 6ms/step - loss: 0.0107 - val_loss: 0.0115
Epoch 11/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 17s 6ms/step - loss: 0.0105 - val_loss: 0.0107
Epoch 12

## NN 4

In [51]:
def create_nn4_model(input_dim, hidden_units1, hidden_units2, hidden_units3, hidden_units4):
    model = Sequential()
    model.add(Dense(hidden_units1, input_dim=input_dim, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))  # Dropout for regularization
    
    model.add(Dense(hidden_units2, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))  # Dropout for regularization
    
    model.add(Dense(hidden_units3, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))  # Dropout for regularization
    
    model.add(Dense(hidden_units4, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))  # Dropout for regularization
    
    model.add(Dense(1))  # Output layer
    return model

def train_nn4_model(X_trn, y_trn, X_vld, y_vld, hidden_units1=32, hidden_units2=16, hidden_units3=8, hidden_units4=4, batch_size=32, epochs=100):
    model = create_nn4_model(X_trn.shape[1], hidden_units1, hidden_units2, hidden_units3, hidden_units4)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    history = model.fit(X_trn, y_trn, 
                        validation_data=(X_vld, y_vld),
                        epochs=epochs,
                        batch_size=batch_size,
                        callbacks=[early_stopping],
                        verbose=1)
    return model, history

In [52]:
# Train the NN4 model
hidden_units1 = 32  # Number of neurons in the first hidden layer
hidden_units2 = 16  # Number of neurons in the second hidden layer
hidden_units3 = 8   # Number of neurons in the third hidden layer
hidden_units4 = 4   # Number of neurons in the fourth hidden layer
NN4_model, NN4_history = train_nn4_model(train_data[features], train_data['RET'], valid_data[features], valid_data['RET'], hidden_units1=hidden_units1, hidden_units2=hidden_units2, hidden_units3=hidden_units3, hidden_units4=hidden_units4)

# Make predictions and evaluate
NN4_predictions = NN4_model.predict(test_data[features])
evaluate(test_data['RET'], NN4_predictions)

Epoch 1/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 26s 6ms/step - loss: 0.5217 - val_loss: 0.0104
Epoch 2/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 19s 6ms/step - loss: 0.0128 - val_loss: 0.0102
Epoch 3/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 19s 6ms/step - loss: 0.0115 - val_loss: 0.0102
Epoch 4/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 18s 6ms/step - loss: 0.0117 - val_loss: 0.0103
Epoch 5/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 19s 6ms/step - loss: 0.0116 - val_loss: 0.0106
Epoch 6/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 19s 6ms/step - loss: 0.0112 - val_loss: 0.0107
Epoch 7/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 19s 6ms/step - loss: 0.0109 - val_loss: 0.0100
Epoch 8/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 19s 6ms/step - loss: 0.0110 - val_loss: 0.0106
Epoch 9/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 18s 6ms/step - loss: 0.0111 - val_loss: 0.0105
Epoch 10/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 19s 6ms/step - loss: 0.0110 - val_loss: 0.0103
Epoch 11/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 19s 6ms/step - loss: 0.0110 - val_loss: 0.0102
Epoch 12

## NN 5

In [53]:
def create_nn5_model(input_dim, hidden_units1, hidden_units2, hidden_units3, hidden_units4, hidden_units5):
    model = Sequential()
    model.add(Dense(hidden_units1, input_dim=input_dim, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))  # Dropout for regularization
    
    model.add(Dense(hidden_units2, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))  # Dropout for regularization
    
    model.add(Dense(hidden_units3, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))  # Dropout for regularization
    
    model.add(Dense(hidden_units4, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))  # Dropout for regularization
    
    model.add(Dense(hidden_units5, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))  # Dropout for regularization
    
    model.add(Dense(1))  # Output layer
    return model

def train_nn5_model(X_trn, y_trn, X_vld, y_vld, hidden_units1=32, hidden_units2=16, hidden_units3=8, hidden_units4=4, hidden_units5=2, batch_size=32, epochs=100):
    model = create_nn5_model(X_trn.shape[1], hidden_units1, hidden_units2, hidden_units3, hidden_units4, hidden_units5)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    history = model.fit(X_trn, y_trn, 
                        validation_data=(X_vld, y_vld),
                        epochs=epochs,
                        batch_size=batch_size,
                        callbacks=[early_stopping],
                        verbose=1)
    return model, history

In [54]:
# Train the NN5 model
hidden_units1 = 32  # Number of neurons in the first hidden layer
hidden_units2 = 16  # Number of neurons in the second hidden layer
hidden_units3 = 8   # Number of neurons in the third hidden layer
hidden_units4 = 4   # Number of neurons in the fourth hidden layer
hidden_units5 = 2   # Number of neurons in the fifth hidden layer
NN5_model, NN5_history = train_nn5_model(train_data[features], train_data['RET'], valid_data[features], valid_data['RET'], hidden_units1=hidden_units1, hidden_units2=hidden_units2, hidden_units3=hidden_units3, hidden_units4=hidden_units4, hidden_units5=hidden_units5)

# Make predictions and evaluate
NN5_predictions = NN5_model.predict(test_data[features])
evaluate(test_data['RET'], NN5_predictions)

Epoch 1/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 29s 6ms/step - loss: 1.6171 - val_loss: 0.0115
Epoch 2/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 19s 6ms/step - loss: 0.0175 - val_loss: 0.0104
Epoch 3/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 21s 7ms/step - loss: 0.0117 - val_loss: 0.0103
Epoch 4/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 22s 7ms/step - loss: 0.0114 - val_loss: 0.0103
Epoch 5/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 20s 7ms/step - loss: 0.0116 - val_loss: 0.0102
Epoch 6/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 20s 7ms/step - loss: 0.0118 - val_loss: 0.0102
Epoch 7/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 21s 7ms/step - loss: 0.0116 - val_loss: 0.0105
Epoch 8/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 20s 7ms/step - loss: 0.0114 - val_loss: 0.0102
Epoch 9/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 20s 7ms/step - loss: 0.0111 - val_loss: 0.0104
Epoch 10/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 21s 7ms/step - loss: 0.0113 - val_loss: 0.0103
Epoch 11/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 20s 7ms/step - loss: 0.0111 - val_loss: 0.0102
Epoch 12

# Portfolios
## Prespecified

In [55]:
# Filter the top 500 firms by 'mve' for each 'DATE'
top_500_data = test_data.sort_values('mve', ascending=False).groupby('DATE').head(500).reset_index(drop=True)

In [56]:
# 1. Calculate the weights for each firm based on 'mve'
top_500_data['weight'] = top_500_data.groupby('DATE')['mve'].transform(lambda x: x / x.sum())

In [57]:
# Predict using the OLS model
top_500_data['OLS_pred'] = OLS_3.predict(top_500_data[features_3])

# Predict using the Huber model
top_500_data['Huber_pred'] = OLS_H_3.predict(top_500_data[features_3])

# Predict using the PLS model
top_500_data['PLS_pred'] = PLS_best_model.predict(top_500_data[features])

# Predict using the PCR model
top_500_data['PCR_pred'] = PCR.predict(top_500_data[features])

# Predict using the NN
top_500_data['NN1_pred'] = NN1_model.predict(top_500_data[features])
top_500_data['NN2_pred'] = NN2_model.predict(top_500_data[features])
top_500_data['NN3_pred'] = NN3_model.predict(top_500_data[features])
top_500_data['NN4_pred'] = NN4_model.predict(top_500_data[features])
top_500_data['NN5_pred'] = NN5_model.predict(top_500_data[features])

1313/1313 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step


In [58]:
monthly_returns = top_500_data.groupby('DATE').apply(
    lambda x: pd.Series({
        'RET': np.average(x['RET'], weights=x['weight']),
        'OLS_pred': np.average(x['OLS_pred'], weights=x['weight']),
        'Huber_pred': np.average(x['Huber_pred'], weights=x['weight']),
        'PLS_pred': np.average(x['PLS_pred'], weights=x['weight']),
        'PCR_pred': np.average(x['PCR_pred'], weights=x['weight']),
        'NN1_pred': np.average(x['NN1_pred'], weights=x['weight']),
        'NN2_pred': np.average(x['NN2_pred'], weights=x['weight']),
        'NN3_pred': np.average(x['NN3_pred'], weights=x['weight']),
        'NN4_pred': np.average(x['NN4_pred'], weights=x['weight']),
        'NN5_pred': np.average(x['NN5_pred'], weights=x['weight']),
    })
).reset_index()

In [59]:
# Define the R_oos function
def R_oos_sp500(actual, predicted):
    actual, predicted = np.array(actual), np.array(predicted).flatten()
    return 1 - (np.dot((actual - predicted), (actual - predicted))) / (np.dot(actual, actual))

# Evaluate for each model
print('*' * 15 + 'R_oos for Models Replicating S&P 500' + '*' * 15)
for model in ['OLS_pred', 'Huber_pred', 'PLS_pred', 'PCR_pred','NN1_pred','NN2_pred','NN3_pred','NN4_pred','NN5_pred']:
    r_oos_value = R_oos_sp500(monthly_returns['RET'], monthly_returns[model])
    print(f'{model}: R_oos = {r_oos_value * 100:.2f}%')

***************R_oos for Models Replicating S&P 500***************
OLS_pred: R_oos = -12.23%
Huber_pred: R_oos = 1.05%
PLS_pred: R_oos = 33.33%
PCR_pred: R_oos = 29.81%
NN1_pred: R_oos = 24.76%
NN2_pred: R_oos = 10.53%
NN3_pred: R_oos = 21.78%
NN4_pred: R_oos = 14.66%
NN5_pred: R_oos = 16.41%


## ML portfolios

In [60]:
# Step 1: Generate predictions for each model
test_data['OLS_pred'] = OLS_3.predict(test_data[features_3])
test_data['Huber_pred'] = OLS_H_3.predict(test_data[features_3])
test_data['PLS_pred'] = PLS_best_model.predict(test_data[features])
test_data['PCR_pred'] = PCR.predict(test_data[features])
test_data['NN1_pred'] = NN1_model.predict(test_data[features])
test_data['NN2_pred'] = NN2_model.predict(test_data[features])
test_data['NN3_pred'] = NN3_model.predict(test_data[features])
test_data['NN4_pred'] = NN4_model.predict(test_data[features])
test_data['NN5_pred'] = NN5_model.predict(test_data[features])

2625/2625 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step
2625/2625 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step
2625/2625 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step
2625/2625 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step
2625/2625 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step


In [61]:
# Step 2: Sort stocks into deciles based on predicted returns
def assign_deciles(pred_col):
    test_data['decile'] = test_data.groupby('DATE')[pred_col].transform(
        lambda x: pd.qcut(x, 10, labels=False, duplicates='drop') + 1
    )

# Assign deciles based on predictions for each model
assign_deciles('OLS_pred')
test_data['OLS_decile'] = test_data['decile']

assign_deciles('Huber_pred')
test_data['Huber_decile'] = test_data['decile']

assign_deciles('PLS_pred')
test_data['PLS_decile'] = test_data['decile']

assign_deciles('PCR_pred')
test_data['PCR_decile'] = test_data['decile']

assign_deciles('NN1_pred')
test_data['NN1_decile'] = test_data['decile']

assign_deciles('NN2_pred')
test_data['NN2_decile'] = test_data['decile']

assign_deciles('NN3_pred')
test_data['NN3_decile'] = test_data['decile']

assign_deciles('NN4_pred')
test_data['NN4_decile'] = test_data['decile']

assign_deciles('NN5_pred')
test_data['NN5_decile'] = test_data['decile']

In [62]:
# Step 3: Calculate simple average returns for each decile and model
def calculate_simple_average_return(data, decile_col):
    # Group by DATE and decile, then calculate mean return for each decile within each month
    return data.groupby(['DATE', decile_col])['RET'].mean().unstack()

# Calculate simple average returns for each decile and model
ols_deciles_returns = calculate_simple_average_return(test_data, 'OLS_decile')
huber_deciles_returns = calculate_simple_average_return(test_data, 'Huber_decile')
pls_deciles_returns = calculate_simple_average_return(test_data, 'PLS_decile')
pcr_deciles_returns = calculate_simple_average_return(test_data, 'PCR_decile')
nn1_deciles_returns = calculate_simple_average_return(test_data, 'NN1_decile')
nn2_deciles_returns = calculate_simple_average_return(test_data, 'NN2_decile')
nn3_deciles_returns = calculate_simple_average_return(test_data, 'NN3_decile')
nn4_deciles_returns = calculate_simple_average_return(test_data, 'NN4_decile')
nn5_deciles_returns = calculate_simple_average_return(test_data, 'NN5_decile')

In [63]:
# Step 4: Construct the Zero-Net-Investment portfolios
def calculate_zni_returns(deciles_returns):
    # Calculate the difference between the top and bottom decile returns
    return deciles_returns[10] - deciles_returns[1]

# Calculate ZNI portfolio returns for each model
ols_zni_returns = calculate_zni_returns(ols_deciles_returns)
huber_zni_returns = calculate_zni_returns(huber_deciles_returns)
pls_zni_returns = calculate_zni_returns(pls_deciles_returns)
pcr_zni_returns = calculate_zni_returns(pcr_deciles_returns)
nn1_zni_returns = calculate_zni_returns(nn1_deciles_returns)
nn2_zni_returns = calculate_zni_returns(nn2_deciles_returns)
nn3_zni_returns = calculate_zni_returns(nn3_deciles_returns)
nn4_zni_returns = calculate_zni_returns(nn4_deciles_returns)
nn5_zni_returns = calculate_zni_returns(nn5_deciles_returns)

In [64]:
# Step 5: Combine and evaluate ZNI portfolios
zni_portfolios = pd.DataFrame({
    'OLS_ZNI': ols_zni_returns,
    'Huber_ZNI': huber_zni_returns,
    'PLS_ZNI': pls_zni_returns,
    'PCR_ZNI': pcr_zni_returns,
    'NN1_ZNI': nn1_zni_returns,
    'NN2_ZNI': nn2_zni_returns,
    'NN3_ZNI': nn3_zni_returns,
    'NN4_ZNI': nn4_zni_returns,
    'NN5_ZNI': nn5_zni_returns
})

In [65]:
# Define the directory path
directory_path = r"C:\Users\frank\OneDrive - Escuela Superior de Economia y Negocios\2. SS 2024\ML Seminar\data\Clean"

# Define the file paths
zni_portfolios_path = os.path.join(directory_path, 'zni_portfolios.csv')
monthly_returns_path = os.path.join(directory_path, 'monthly_returns.csv')

# Save the DataFrames to CSV files
zni_portfolios.to_csv(zni_portfolios_path, index=True)  # index=True to include the index in the CSV
monthly_returns.to_csv(monthly_returns_path, index=True)  # index=True to include the index in the CSV

In [66]:
# Ensure `monthly_returns` has a 'DATE' column and 'RET' column
monthly_returns = monthly_returns.reset_index()  # Reset index to ensure 'DATE' is a column
monthly_returns.rename(columns={'RET': 'SP500_RET'}, inplace=True)

# Merge `monthly_returns` into `zni_portfolios` on 'DATE'
zni_portfolios = zni_portfolios.reset_index()  # Reset index to ensure 'DATE' is a column
merged_portfolios = pd.merge(zni_portfolios, monthly_returns[['DATE', 'SP500_RET']], on='DATE', how='left')

print(merged_portfolios.head())  # Display the first few rows for verification

        DATE   OLS_ZNI  Huber_ZNI   PLS_ZNI   PCR_ZNI   NN1_ZNI   NN2_ZNI  \
0 2013-01-31  0.016302   0.024629  0.027263 -0.005990  0.019370 -0.003016   
1 2013-02-28  0.036944   0.029284 -0.008795  0.016225 -0.040705 -0.016622   
2 2013-03-31  0.039756   0.032568 -0.010146 -0.023190 -0.029756 -0.041115   
3 2013-04-30 -0.016158  -0.018293 -0.030515 -0.018367 -0.038761 -0.016134   
4 2013-05-31  0.001337   0.001451  0.034203 -0.005684  0.061622  0.042626   

    NN3_ZNI   NN4_ZNI   NN5_ZNI  SP500_RET  
0  0.018499  0.006361  0.022363   0.058423  
1 -0.029221  0.019581 -0.027901   0.006047  
2 -0.042903 -0.025832 -0.031605   0.038767  
3 -0.033762 -0.008265 -0.039524   0.018049  
4  0.069833  0.017463  0.042039   0.018000  


Calculate the Sharpe Ratio for each portfolio

In [69]:
def calculate_sharpe_ratio(df, column_name):
    """Calculate the Sharpe Ratio for a given column in the DataFrame."""
    mean_return = df[column_name].mean()
    std_dev = df[column_name].std()
    return mean_return / std_dev if std_dev != 0 else np.nan

# Calculate Sharpe Ratio for each portfolio
sharpe_ratios = {col: calculate_sharpe_ratio(merged_portfolios, col) for col in merged_portfolios.columns if col.endswith('_ZNI')}

# Calculate Sharpe Ratio for S&P500
sharpe_ratios['SP500'] = calculate_sharpe_ratio(merged_portfolios, 'SP500_RET')

# Print the Sharpe Ratios
print("Sharpe Ratios:")
for portfolio, ratio in sharpe_ratios.items():
    print(f"{portfolio}: {ratio:.2f}")

Sharpe Ratios:
OLS_ZNI: -0.06
Huber_ZNI: -0.07
PLS_ZNI: 0.06
PCR_ZNI: 0.14
NN1_ZNI: -0.04
NN2_ZNI: 0.15
NN3_ZNI: 0.09
NN4_ZNI: 0.12
NN5_ZNI: -0.01
SP500: 0.35


Calculate and test $\alpha$

In [68]:
import statsmodels.api as sm

def calculate_alpha_beta(portfolio_returns, market_returns):
    """Calculate alpha and beta from a regression."""
    X = sm.add_constant(market_returns)  # Add a constant for the intercept
    model = sm.OLS(portfolio_returns, X).fit()  # Fit the OLS model
    alpha = model.params[0]  # Intercept
    beta = model.params[1]   # Slope
    p_value = model.pvalues[0]  # P-value for alpha
    return alpha, beta, p_value

# Calculate alpha and beta for each portfolio and the S&P500
results = {}
for portfolio in [col for col in merged_portfolios.columns if col.endswith('_ZNI')]:
    portfolio_returns = merged_portfolios[portfolio]
    market_returns = merged_portfolios['SP500_RET']
    alpha, beta, p_value = calculate_alpha_beta(portfolio_returns, market_returns)
    results[portfolio] = {'alpha': alpha, 'beta': beta, 'p_value': p_value}

# Print the results
print("\nAlpha and Beta:")
for portfolio, metrics in results.items():
    alpha_significance = 'Significant' if metrics['p_value'] < 0.05 else 'Not Significant'
    print(f"{portfolio}: Alpha = {metrics['alpha']:.4f}, Beta = {metrics['beta']:.4f}, Alpha p-value = {metrics['p_value']:.4f} ({alpha_significance})")


Alpha and Beta:
OLS_ZNI: Alpha = -0.0003, Beta = -0.1353, Alpha p-value = 0.9206 (Not Significant)
Huber_ZNI: Alpha = -0.0008, Beta = -0.0858, Alpha p-value = 0.7801 (Not Significant)
PLS_ZNI: Alpha = 0.0030, Beta = -0.0927, Alpha p-value = 0.3975 (Not Significant)
PCR_ZNI: Alpha = 0.0052, Beta = -0.1560, Alpha p-value = 0.0796 (Not Significant)
NN1_ZNI: Alpha = -0.0036, Beta = 0.2094, Alpha p-value = 0.3659 (Not Significant)
NN2_ZNI: Alpha = 0.0044, Beta = 0.0111, Alpha p-value = 0.2235 (Not Significant)
NN3_ZNI: Alpha = 0.0025, Beta = 0.0683, Alpha p-value = 0.5676 (Not Significant)
NN4_ZNI: Alpha = 0.0060, Beta = -0.2605, Alpha p-value = 0.0423 (Significant)
NN5_ZNI: Alpha = -0.0018, Beta = 0.1323, Alpha p-value = 0.6389 (Not Significant)


In [71]:
# Convert alpha and beta results to a DataFrame
alpha_beta_df = pd.DataFrame.from_dict(results, orient='index').reset_index()
alpha_beta_df.rename(columns={'index': 'Portfolio'}, inplace=True)

# Save alpha and beta results to CSV
alpha_beta_df.to_csv(r'C:\Users\frank\OneDrive - Escuela Superior de Economia y Negocios\2. SS 2024\ML Seminar\Article redaction\Tables\alpha_beta_results.csv', index=False)